In [21]:
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import numpy as np
import tensorflow as tf

In [54]:
import pandas as pd

# CSV-Datei einlesen und Header überspringen
df = pd.read_csv('/Users/riccardo/Desktop/Repositorys_Github/Training/Dataset/housing.csv', sep=',')
df
# Ndfun enthält der DataFrame keine Spaltennamen


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
9994,-121.05,39.13,10.0,3063.0,497.0,1168.0,507.0,4.4375,185100.0,INLAND
9995,-121.03,39.14,10.0,3138.0,524.0,1275.0,511.0,4.0775,164500.0,INLAND
9996,-121.07,39.13,8.0,4839.0,832.0,1977.0,762.0,4.0848,155900.0,INLAND
9997,-121.05,39.11,7.0,2767.0,423.0,1143.0,382.0,3.6333,170200.0,INLAND


In [53]:
df.shape

(112457, 6)

In [55]:
df_mod = df[[ 'housing_median_age', 'population', 
             'households', 'median_income',
              'median_house_value']]


In [56]:
# Daten in Features (X) und Labels (y) aufteilen
X = df_mod.drop(columns=['median_house_value'])  # Features ohne die Zielvariable
y = df_mod[["median_house_value"]]  # Nur die Zielvariable


In [57]:
# Zuerst die Daten in Trainings- und Testsets aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dann die Testdaten in Test- und Validierungsdaten aufteilen
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)



In [58]:
print("Der X Test Datensatz besthet aus: ", X_train.shape, "Der Y Test Datensatz besthet aus: ", y_train.shape)
print("Der X_test Datensatz besthet aus: ", X_test.shape,"Der y_test Datensatz besteht  aus: ", y_test.shape)

Der X Test Datensatz besthet aus:  (7999, 4) Der Y Test Datensatz besthet aus:  (7999, 1)
Der X_test Datensatz besthet aus:  (1000, 4) Der y_test Datensatz besteht  aus:  (1000, 1)


## Erstellungs des RNNs Infrastruktur

In [65]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
# Überprüfen der Form der Trainingsdaten X_train
print("Form der Trainingsdaten X_train:", X_train.shape)

# Definition des Modells mit der richtigen input_shape
#Initialize RNN:
regressor = Sequential()

#Adding the first RNN layer and some Dropout regularization
regressor.add(SimpleRNN(units = 50, activation='tanh', return_sequences=True, input_shape= (X_train.shape[1])))

Form der Trainingsdaten X_train: (7999, 4)


TypeError: 'int' object is not iterable

In [63]:
regressor.compile(loss='mean_squared_error', optimizer='adam')


In [64]:
regressor.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20


2024-03-01 16:11:27.541182: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Incompatible shapes: [32,1] vs. [32,4,50]
	 [[{{node mean_squared_error/SquaredDifference}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/Users/riccardo/anaconda3/envs/model/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
     

In [40]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                256       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.evaluate(X_test, y_test)

 1/32 [..............................] - ETA: 0s - loss: 62603083776.0000

32/32 [==============================] - 0s 390us/step - loss: 55479328768.0000


55479328768.0

1/1 [==============================] - 0s 14ms/step


array([[94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595],
       [94.916595]], dtype=float32)

### Model 2

In [36]:

model_SimpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=(X_train.shape[1],))
])

ValueError: Input 0 of layer "simple_rnn_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 7)